In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Test").getOrCreate()

print(spark.version)


3.5.4


In [1]:
from pyspark import SparkContext

In [3]:
sc = SparkContext()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/06 21:41:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 50039)
Traceback (most recent call last):
  File "/opt/anaconda3/envs/pyspark_env/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/anaconda3/envs/pyspark_env/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/anaconda3/envs/pyspark_env/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/anaconda3/envs/pyspark_env/lib/python3.9/socketserver.py", line 747, in __init__
    self.handl

In [9]:
%%writefile example.txt
first line
second line
third line
forth line

Writing example.txt


In [11]:
# Create RDD using a textFile method
textFile=sc.textFile('example.txt')

## PySpark RDD Transformations & Actions

* Transformations create a new RDD from an existing one.
* Actions trigger execution and return results to the driver.
* RDDs are immutable, meaning transformations create a new RDD.
---

# RDD Transformations

| **Transformation**    | **Description** |
|----------------------|----------------|
| `map(func)`         | Applies `func` to each element and returns a new RDD |
| `flatMap(func)`     | Similar to `map()`, but flattens nested lists |
| `filter(func)`      | Filters elements based on `func` |
| `groupByKey()`      | Groups data by key (used for key-value pairs) |
| `reduceByKey(func)` | Applies `func` to reduce values for each key |
| `sortByKey()`       | Sorts the RDD by key |
| `join(rdd2)`        | Joins two RDDs based on keys |
| `coalesce(n)`       | Reduces the number of partitions |
| `distinct()`        | Removes duplicate values |


## RDD Actions

In [14]:
textFile.count()

4

In [16]:
textFile.first()

'first line'

## RDD Transformation

### `filter()`: Filter specific values


In [19]:
# step 1 transformation
secfind=textFile.filter(lambda line:'line' in line)

In [21]:
# step 2 action
secfind.collect()

['first line', 'second line', 'third line', 'forth line']

In [148]:
# Filter transactions greater than 200
filtered_rdd = kv_rdd.filter(lambda x: x[1] > 200)
print("Filtered RDD:", filtered_rdd.collect())


Filtered RDD: [('129', 450.0), ('131', 300.0)]


### `groupByKey()`: Group data by key

In [151]:
# Group transactions by ServiceID
grouped_rdd = kv_rdd.groupByKey().mapValues(list)
print("Grouped RDD:", grouped_rdd.collect())

Grouped RDD: [('121', [200.0]), ('122', [150.0]), ('131', [100.0, 300.0]), ('129', [450.0])]


In [23]:
secfind.count()

4

In [1]:
%%writefile example2.txt
first 
second line
the third line
then a fourth line

Writing example2.txt


In [7]:
from pyspark import SparkContext

In [9]:
sc = SparkContext()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/12 23:35:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
# Show RDD
sc.textFile('example2.txt')

example2.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [13]:
# Save a reference to this RDD
text_rdd = sc.textFile('example2.txt')

In [13]:
# Map a function (or lambda expression) to each line
# Then collect the results.
text_rdd.map(lambda line: line.split()).collect()

[['first'],
 ['second', 'line'],
 ['the', 'third', 'line'],
 ['then', 'a', 'fourth', 'line']]

In [19]:
word = text_rdd.map(lambda line: line.split())

In [21]:
word.collect()

[['first'],
 ['second', 'line'],
 ['third', 'line'],
 ['the', 'a', 'forth', 'line']]

In [23]:
# Output is
text_rdd.collect()

['first', 'second line', 'third line', 'the a forth line']

`flatMap()`: Flatten nested structures

In [140]:
# Split rows into words
words_rdd = rdd.flatMap(lambda line: line.split())
print("FlatMapped RDD:", words_rdd.take(10))


FlatMapped RDD: ['201', '10/13/2017', '100', 'NY', '131', '100.00', '204', '10/18/2017', '700', 'TX']


In [ ]:
# Split rows into words
words_rdd = rdd.flatMap(lambda line: line.split())
print("FlatMapped RDD:", words_rdd.take(10))


In [15]:
# Collect everything as a single flat map
text_rdd.flatMap(lambda line: line.split()).collect()

['first',
 'second',
 'line',
 'the',
 'third',
 'line',
 'then',
 'a',
 'fourth',
 'line']

## RDDs and Key Value Pairs


In [18]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Writing services.txt


In [21]:
services = sc.textFile('services.txt')

In [25]:
# First 2 elements of RDD
services.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

`map()`: Apply function to each row

In [134]:
# Extract ServiceID and Amount as key-value pairs
kv_rdd = rdd.map(lambda line: (line.split()[4], float(line.split()[-1])))
print("Mapped RDD:", kv_rdd.collect())

Mapped RDD: [('131', 100.0), ('129', 450.0), ('121', 200.0), ('131', 300.0), ('122', 150.0)]


In [27]:
services.map(lambda x: x.split())

PythonRDD[11] at RDD at PythonRDD.scala:53

In [31]:
services.map(lambda x: x.split()).collect()

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [29]:
services.map(lambda x: x.split()).take(3)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

In [39]:
%%writefile data2.txt
#ID    Name    Age    City    Score
1      John    25     NY      85.5
2      Jane    30     TX      92.0
3      Alice   27     CA      88.0
4      Bob     24     FL      79.5
5      Eve     29     WA      95.0


Writing data.txt


In [35]:
# Removing hash tag
services.map(lambda x: x[1:] if x[0]=='#' else x).collect()

['EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00',
 '204       10/18/2017      700       TX       129          450.00',
 '202       10/15/2017      203       CA       121          200.00',
 '206       10/19/2017      202       CA       131          500.00',
 '203       10/17/2017      101       NY       173          750.00',
 '205       10/19/2017      202       TX       121          200.00']

In [37]:
services.map(lambda x: x[1:] if x[0]=='#' else x).map(lambda x: x.split()).collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

Here's your **Jupyter Notebook Markdown tutorial** for **RDD Actions** in PySpark. It includes a **pretext table** explaining each action and the corresponding **Python code** in proper markdown formatting.

---

### **RDD Actions in PySpark**  

#### **Action Description Table**
| Action             | Description |
|--------------------|------------|
| `count()`         | Counts total rows |
| `first()`         | Retrieves first row |
| `collect()`       | Converts RDD to list |
| `take(n)`        | Fetches first `n` rows |
| `distinct()`      | Returns distinct rows |
| `reduce()`        | Applies a function to aggregate values |
| `max()`           | Finds the maximum value |
| `min()`           | Finds the minimum value |
| `countByKey()`    | Counts occurrences of each key |
| `keys()`          | Retrieves all keys in an RDD |
| `values()`        | Retrieves all values in an RDD |
| `isEmpty()`       | Checks if RDD is empty |
| `takeSample(False, n)` | Takes `n` random samples |

---



In [82]:

%%writefile data.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
205       10/20/2017      404       FL       131          300.00
203       10/17/2017      305       TX       122          150.00

Overwriting data.txt


In [84]:
# To check the saved file, run:
import os
print(os.getcwd())  # Shows current working directory
print(os.listdir())  # Lists files in the directory


/Users/apple/Documents/GitHub/Python/BigData
['services.txt', '.DS_Store', 'example2.txt', 'lambda_expressions.ipynb', 'example.txt', '.ipynb_checkpoints', 'pyspark.ipynb', 'data.txt']


In [86]:
# Load data.txt into an RDD (excluding header)
rdd = sc.textFile("data.txt").filter(lambda line: not line.startswith("#"))

# Display first few lines
rdd.take(3)

['201       10/13/2017      100       NY       131          100.00',
 '204       10/18/2017      700       TX       129          450.00',
 '202       10/15/2017      203       CA       121          200.00']

## RDD Actions in PySpark
- Actions are operations that **trigger computation and return values to the driver**.
### **Basic Actions**

In [88]:
# Count number of records
print("Total rows:", rdd.count())



Total rows: 5


In [90]:
# Show the first row
print("First row:", rdd.first())



First row: 201       10/13/2017      100       NY       131          100.00


In [92]:
# Collect all records into a list
print("All records:", rdd.collect())



All records: ['201       10/13/2017      100       NY       131          100.00', '204       10/18/2017      700       TX       129          450.00', '202       10/15/2017      203       CA       121          200.00', '205       10/20/2017      404       FL       131          300.00', '203       10/17/2017      305       TX       122          150.00']


In [94]:
# Take the first 3 rows
print("First 3 rows:", rdd.take(3))




First 3 rows: ['201       10/13/2017      100       NY       131          100.00', '204       10/18/2017      700       TX       129          450.00', '202       10/15/2017      203       CA       121          200.00']


In [96]:
# Retrieve distinct rows
print("Distinct rows:", rdd.distinct().collect())


Distinct rows: ['204       10/18/2017      700       TX       129          450.00', '202       10/15/2017      203       CA       121          200.00', '205       10/20/2017      404       FL       131          300.00', '203       10/17/2017      305       TX       122          150.00', '201       10/13/2017      100       NY       131          100.00']


### **Numeric Operations**

In [103]:

# Extract the "Amount" column (last column) and convert to float
amount_rdd = rdd.map(lambda line: float(line.split()[-1]))

In [105]:
# Find the total sum of Amount
total_amount = amount_rdd.reduce(lambda x, y: x + y)
print("Total Amount:", total_amount)


Total Amount: 1200.0


In [107]:

# Find the maximum transaction amount
max_amount = amount_rdd.max()
print("Max Amount:", max_amount)

Max Amount: 450.0


In [109]:
# Find the minimum transaction amount
min_amount = amount_rdd.min()
print("Min Amount:", min_amount)

Min Amount: 100.0


In [111]:
# Compute average amount
avg_amount = total_amount / amount_rdd.count()
print("Average Amount:", avg_amount)

Average Amount: 240.0


### **Some other Useful Actions**

In [114]:
# Get the first 2 records as key-value pairs (ServiceID, Amount)
kv_rdd = rdd.map(lambda line: (line.split()[4], float(line.split()[-1])))
print("Key-Value pairs:", kv_rdd.take(2))

Key-Value pairs: [('131', 100.0), ('129', 450.0)]


In [116]:
# Count by key (ServiceID)
service_count = kv_rdd.countByKey()
print("Count per ServiceID:", dict(service_count))

Count per ServiceID: {'131': 2, '129': 1, '121': 1, '122': 1}


In [118]:
# Fetch all keys (ServiceIDs)
print("All ServiceIDs:", kv_rdd.keys().collect())

All ServiceIDs: ['131', '129', '121', '131', '122']


In [120]:
# Fetch all values (Amounts)
print("All Amounts:", kv_rdd.values().collect())

All Amounts: [100.0, 450.0, 200.0, 300.0, 150.0]


In [122]:
# Check if RDD is empty
print("Is RDD empty?", rdd.isEmpty())

Is RDD empty? False


In [124]:
# Take a sample (without replacement)
print("Sample Records:", rdd.takeSample(False, 2))

Sample Records: ['201       10/13/2017      100       NY       131          100.00', '204       10/18/2017      700       TX       129          450.00']
